function ClickConnect(){
console.log("Working");
document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click();
}
setInterval(ClickConnect,60000)

## Base Line

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
# load csv
train_path = "/content/drive/Shareddrives/AI_CUP_NLP/Batch_answers - train_data (no-blank).csv"
train_data = pd.read_csv(train_path, encoding = "utf-8", index_col='id').iloc[:,:5].applymap(lambda x: x.strip('"')).reset_index()

In [ ]:
train_data["length"] = train_data["q'"] + train_data["r'"]
train_data["length"] = train_data["length"].map(len)
idx = train_data.groupby(by=train_data.id)['length'].transform(max) == train_data['length']
small_train_data = train_data[idx].reset_index(drop=True)
small_train_data = small_train_data.groupby(by=small_train_data.id).first().reset_index(drop=True)

In [ ]:
# 評分用LCS
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize
punctuations = '''!"#$%&'()*+, -./:;<=>?@[\]^_`{|}~'''

def longestCommonSubsequence(text1: str, text2: str) -> int:

    text1 = [i for i in word_tokenize(text1) if len(i)>1 or i not in punctuations]
    text2 = [i for i in word_tokenize(text2) if len(i)>1 or i not in punctuations]

    if len(text2) > len(text1):
        text1, text2 = text2, text1
    lcs = [[0]*(len(text2)+1) for _ in range(2)]
    for i in range(1, len(text1)+1):
        for j in range(1, len(text2)+1):
            if text1[i-1]== text2[j-1]:
                lcs[i%2][j] = lcs[(i-1) % 2][j-1] +1
            else:
                lcs[i%2][j]= max(lcs[(i-1)%2][j], lcs[i % 2][j-1])
    lcs = lcs[len(text1)% 2][len(text2)]
    return lcs/(len(text1)+len(text2)-lcs)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
from tqdm import tqdm

In [ ]:
q_score_list = []
r_score_list = []

for i in tqdm(small_train_data.index):
  try:
    q_score = longestCommonSubsequence(small_train_data['q'][i], small_train_data["q'"][i])
    r_score = longestCommonSubsequence(small_train_data['r'][i], small_train_data["r'"][i])
  except:
    q_score = -1
    r_score = -1
  
  q_score_list.append(q_score)
  r_score_list.append(r_score)

small_train_data['q_score'] = q_score_list
small_train_data['r_score'] = r_score_list

100%|██████████| 7987/7987 [00:45<00:00, 173.82it/s]


In [ ]:
sum(small_train_data['q_score']==-1), sum(small_train_data['r_score']==-1)

(0, 0)

In [ ]:
q_base_line = small_train_data['q_score'].sum() / small_train_data['q_score'].count()
r_base_line = small_train_data['r_score'].sum() / small_train_data['r_score'].count()

q_base_line, r_base_line, (q_base_line+r_base_line)/2

(0.7111274291681386, 0.6609471793216652, 0.686037304244902)

## Test Data

In [ ]:
test_df = pd.read_csv('/content/drive/Shareddrives/AI_CUP_NLP/Batch_answers - test_data(no_label).csv', index_col='id')

In [ ]:
fake_answer = test_df.drop('s', axis=1)
fake_answer.to_csv('/content/drive/Shareddrives/AI_CUP_NLP/fake_answer.csv')